In [95]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import ee
import requests
import numpy as np
from numpy.lib.recfunctions import structured_to_unstructured
import io
import time
from typing import Tuple

In [80]:
PATCH_SIZE = 128

In [81]:
project = "bigdata-ahhcash"
def initialize_ee():
  ee.Authenticate()
  ee.Initialize(project=project, opt_url="https://earthengine-highvolume.googleapis.com")

In [82]:
initialize_ee()

In [110]:
def get_landsat_image(date: datetime = datetime.now() - relativedelta(years=5)) -> ee.Image:
    """Gets a Landsat 8 image for the selected date."""
    start_date = ee.Date(date)
    end_date = ee.Date(date).advance(6, 'month')
    return (
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterDate(start_date, end_date)
        .mosaic()
    )

def get_landsat_ndvi(image: ee.Image) -> ee.Image:
    """Calculates NDVI from a Landsat 8 image."""
    return image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")

In [111]:
def get_modis_ndvi(date: datetime) -> ee.Image:
    """Gets MODIS NDVI data for a given date."""
    start_date = ee.Date(date)
    end_date = ee.Date(date).advance(1, 'month')
    return (
        ee.ImageCollection("MODIS/061/MOD13Q1")
        .filterDate(start_date, end_date)
        .select("NDVI")
        .first()
    )

In [112]:
def get_landsat_lst(image: ee.Image) -> ee.Image:
    """
    Calculates Land Surface Temperature from a Landsat 8 image.
    This function is based on the formula in the following page https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2
    """
    return image.select("ST_B10").multiply(0.00341802).add(149.0).rename("LST")

In [114]:
def get_inputs_image(date: datetime = datetime.now() - relativedelta(years=4)) -> ee.Image:
    """Gets an Earth Engine image with all the inputs for the model."""
    # Get MODIS NDVI
    modis_ndvi = get_modis_ndvi(date)

    # Get Landsat data
    landsat_image = get_landsat_image(date)
    landsat_ndvi = get_landsat_ndvi(landsat_image)
    landsat_lst = get_landsat_lst(landsat_image)

    combined_ndvi = ee.Image.cat([landsat_ndvi, modis_ndvi])

    # Combine all input data
    return ee.Image([combined_ndvi, landsat_lst])

In [115]:
img = get_inputs_image()
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'NDVI_1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [172800, 72000],
   'crs': 'SR-ORG:6974',
   'crs_transform': [231.65635826395825,
    0,
    -20015109.354,
    0,
    -231.65635826395834,
    10007554.677003]},
  {'id': 'LST',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 149,
    'max': 372.9999407},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [98]:
def get_patch(
    image: ee.Image, lonlat: Tuple[float, float], patch_size: int, scale: int
) -> np.ndarray:
    """Fetches a patch of pixels from Earth Engine."""
    point = ee.Geometry.Point(lonlat)
    url = image.getDownloadURL(
        {
            "region": point.buffer(scale * patch_size / 2, 1).bounds(1),
            "dimensions": [patch_size, patch_size],
            # "scale": SCALE,
            "format": "NPY",
        }
    )

    # Retry on "Too Many Requests" errors
    response = requests.get(url)
    if response.status_code == 429:
        raise Exception("Too Many Requests")

    # Raise other exceptions
    response.raise_for_status()
    return np.load(io.BytesIO(response.content), allow_pickle=True)

In [118]:
def get_inputs_patch(
    date: datetime = datetime.now() - relativedelta(years = 1),
    lonlat: tuple[float, float] = (-140.0, 60.0),
    patch_size: int = 128,
) -> np.ndarray:
    """Gets the inputs patch of pixels for the given point and date."""
    image = get_inputs_image(date)
    patch = get_patch(image, lonlat, patch_size, 128)
    return structured_to_unstructured(patch)

In [120]:
initialize_ee()
ip = get_inputs_patch()
ip.shape

(128, 128, 3)

In [108]:
from __future__ import annotations

import numpy as np
from plotly.graph_objects import Image
from plotly.subplots import make_subplots

In [18]:
CLASSIFICATIONS = {
    "💧 Water": "419BDF",
    "🌳 Trees": "397D49",
    "🌾 Grass": "88B053",
    "🌿 Flooded vegetation": "7A87C6",
    "🚜 Crops": "E49635",
    "🪴 Shrub and scrub": "DFC35A",
    "🏗️ Built-up areas": "C4281B",
    "🪨 Bare ground": "A59B8F",
    "❄️ Snow and ice": "B39FE1",
}

In [19]:
def render_rgb_images(
    values: np.ndarray, min: float = 0.0, max: float = 1.0
) -> np.ndarray:
    """Renders a numeric NumPy array with shape (width, height, rgb) as an image.

    Args:
        values: A float array with shape (width, height, rgb).
        min: Minimum value in the values.
        max: Maximum value in the values.

    Returns: An uint8 array with shape (width, height, rgb).
    """
    scaled_values = (values - min) / (max - min)
    rgb_values = np.clip(scaled_values, 0, 1) * 255
    return rgb_values.astype(np.uint8)

In [21]:
def render_classifications(values: np.ndarray, palette: list[str]) -> np.ndarray:
    """Renders a classifications NumPy array with shape (width, height, 1) as an image.

    Args:
        values: An uint8 array with shape (width, height, 1).
        palette: List of hex encoded colors.

    Returns: An uint8 array with shape (width, height, rgb) with colors from the palette.
    """
    # Create a color map from a hex color palette.
    xs = np.linspace(0, len(palette), 256)
    indices = np.arange(len(palette))

    red = np.interp(xs, indices, [int(c[0:2], 16) for c in palette])
    green = np.interp(xs, indices, [int(c[2:4], 16) for c in palette])
    blue = np.interp(xs, indices, [int(c[4:6], 16) for c in palette])

    color_map = np.array([red, green, blue]).astype(np.uint8).transpose()
    color_indices = (values / len(palette) * 255).astype(np.uint8)
    return np.take(color_map, color_indices, axis=0)

In [22]:
def render_sentinel2(patch: np.ndarray, max: float = 3000) -> np.ndarray:
    """Renders a Sentinel 2 image."""
    red = patch[:, :, 3]  # B4
    green = patch[:, :, 2]  # B3
    blue = patch[:, :, 1]  # B2
    rgb_patch = np.stack([red, green, blue], axis=-1)
    return render_rgb_images(rgb_patch, 0, max)

In [23]:
def render_landcover(patch: np.ndarray) -> np.ndarray:
    """Renders a land cover image."""
    palette = list(CLASSIFICATIONS.values())
    return render_classifications(patch[:, :, 0], palette)

In [24]:
def show_inputs(inputs: np.ndarray, max: float = 3000) -> None:
    """Shows the input data as an image."""
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Sentinel 2"))
    fig.add_trace(Image(z=render_sentinel2(inputs, max)), row=1, col=1)
    fig.show()

In [25]:
def show_outputs(outputs: np.ndarray) -> None:
    """Shows the outputs/labels data as an image."""
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Land cover",))
    fig.add_trace(Image(z=render_landcover(outputs)), row=1, col=1)
    fig.show()

In [26]:
def show_example(inputs: np.ndarray, labels: np.ndarray, max: float = 3000) -> None:
    """Shows an example of inputs and labels an image."""
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Sentinel 2", "Land cover"))
    fig.add_trace(Image(z=render_sentinel2(inputs, max)), row=1, col=1)
    fig.add_trace(Image(z=render_landcover(labels)), row=1, col=2)
    fig.show()

In [27]:
def show_legend() -> None:
    """Shows the legend of the land cover classifications."""

    def color_box(red: int, green: int, blue: int) -> str:
        return f"\033[48;2;{red};{green};{blue}m"

    reset_color = "\u001b[0m"
    for name, color in CLASSIFICATIONS.items():
        red = int(color[0:2], 16)
        green = int(color[2:4], 16)
        blue = int(color[4:6], 16)
        print(f"{color_box(red, green, blue)}   {reset_color} {name}")

In [121]:
show_inputs(ip)

IndexError: index 3 is out of bounds for axis 2 with size 3